<a href="https://colab.research.google.com/github/devadiga-navya/widsDatathon_2024/blob/main/WiDS_2024_1_%7C_3rd_Place_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'widsdatathon2024-challenge1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F65862%2F7469115%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240302%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240302T074401Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D698afa538a2e5b7b6bcc9a0877afce4b4d67290f8046f6dfd97918de96ba302f12d19c4687d18445b242d753f969fc2d69674e411d2e524b655c3aacf55a8d77e988ff0fc2fdfaea8114737a7c3591559429c0b9939a1c08e40ae7a0735ae5e110f0027ab52f0ca1e01fbd523f6da832837868bf877f33d990ab585864a71cbd0d9e5e5037b7e3b1c871cd0afb21094cf702f246d06e4346943508c43cfdd38214b350351d5c0e827fe035866dc808979b19745d53edb64d02cf483ec533a32ee763ad9d13f87475a1b06fe07b052df0503200982f8d481d49c714fec0ebf2b75d0e0a339e5450be9c07534c47f9254b76a01990217cdd941e17463e887cd399'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Loading Libraries and Datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import optuna
import re

from category_encoders import OneHotEncoder, MEstimateEncoder, CatBoostEncoder
from sklearn import set_config
from sklearn.inspection import permutation_importance
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, f1_score, accuracy_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone, ClassifierMixin
from sklearn.preprocessing import FunctionTransformer, StandardScaler, PowerTransformer, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Perceptron, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')

pd.set_option('display.max_rows', 100)
set_config(transform_output = 'pandas')
pd.options.mode.chained_assignment = None
#optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
train = pd.read_csv(r'/kaggle/input/widsdatathon2024-challenge1/training.csv', index_col = 'patient_id')
test = pd.read_csv(r'/kaggle/input/widsdatathon2024-challenge1/test.csv', index_col = 'patient_id')

# Descriptive Statistics

Let's begin by taking a peek at our training dataset first

In [ ]:
train.head(10)

In [ ]:
desc = pd.DataFrame(index = list(train))
desc['type'] = train.dtypes
desc['count'] = train.count()
desc['nunique'] = train.nunique()
desc['%unique'] = desc['nunique'] / len(train) * 100
desc['null'] = train.isnull().sum()
desc['%null'] = desc['null'] / len(train) * 100
#desc['min'] = train.min()
#desc['max'] = train.max()
desc

In [ ]:
test.head(10)

In [ ]:
desc = pd.DataFrame(index = list(test))
desc['type'] = test.dtypes
desc['count'] = test.count()
desc['nunique'] = test.nunique()
desc['%unique'] = desc['nunique'] / len(test) * 100
desc['null'] = test.isnull().sum()
desc['%null'] = desc['null'] / len(test) * 100
#desc['min'] = test.min()
#desc['max'] = test.max()
desc

In [ ]:
numerical_features = list(test._get_numeric_data())
categorical_features = list(test.select_dtypes('object'))

# Adversarial Validation

In [ ]:
def adversarial_validation(dataset_1 = train, dataset_2 = test, target = [], label = 'Train-Test'):

    adv_train = dataset_1.drop(target, axis = 1)
    adv_test = dataset_2.copy()

    adv_train['is_test'] = 0
    adv_test['is_test'] = 1

    adv = pd.concat([adv_train, adv_test], ignore_index = True)

    adv_shuffled = adv.sample(frac = 1)

    adv_X = adv_shuffled.drop('is_test', axis = 1)
    adv_y = adv_shuffled.is_test

    skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

    val_scores = []
    predictions = np.zeros(len(adv))

    for fold, (train_idx, val_idx) in enumerate(skf.split(adv_X, adv_y)):

        adv_lr = make_pipeline(MEstimateEncoder(cols = categorical_features), XGBClassifier(random_state = 42))
        adv_lr.fit(adv_X.iloc[train_idx], adv_y.iloc[train_idx])

        val_preds = adv_lr.predict_proba(adv_X.iloc[val_idx])[:,1]
        predictions[val_idx] = val_preds
        val_score = roc_auc_score(adv_y.iloc[val_idx], val_preds)
        val_scores.append(val_score)

    fpr, tpr, _ = roc_curve(adv['is_test'], predictions)

    plt.figure(figsize = (10, 10), dpi = 300)
    sns.lineplot(x=[0, 1], y=[0, 1], linestyle="--", label="Indistinguishable Datasets")
    sns.lineplot(x=fpr, y=tpr, label="Adversarial Validation Classifier")
    plt.title(f'{label} Validation = {np.mean(val_scores):.5f}', weight = 'bold', size = 17)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()

In [ ]:
#adversarial_validation()

# Corr

In [ ]:
def heatmap(dataset, label = None):
    corr = dataset.corr(method = 'spearman')
    plt.figure(figsize = (25, 25), dpi = 300)
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(corr, mask = mask, cmap = 'viridis', annot = True, annot_kws = {'size' : 5})
    plt.title(f'{label} Dataset Correlation Matrix\n', fontsize = 25, weight = 'bold')
    plt.show()

In [ ]:
#heatmap(train[numerical_features])

In [ ]:
def distance(data, label = ''):
    #thanks to @sergiosaharovsky for the fix
    corr = data.corr(method = 'spearman')
    dist_linkage = linkage(squareform(1 - abs(corr)), 'complete')

    plt.figure(figsize = (10, 8), dpi = 300)
    dendro = dendrogram(dist_linkage, labels=data.columns, leaf_rotation=90)
    plt.title(f'Feature Distance in {label} Dataset', weight = 'bold', size = 20)
    plt.show()

In [ ]:
#distance(train[numerical_features])

# Preparation

This is where we start preparing everything if we want to start building machine learning models.

In [ ]:
X = train.drop('patient_gender', axis = 1)
y = X.pop('DiagPeriodL90D')

roc_scorer = make_scorer(roc_auc_score, needs_proba = True)

seed = 42
splits = 5
skf = StratifiedKFold(n_splits = splits, random_state = seed, shuffle = True)
tf.keras.utils.set_random_seed(seed)

# Feature Engineering

In [ ]:
class FeatureDropper(BaseEstimator, TransformerMixin):

    def __init__(self, cols):
        self.cols = cols

    def fit(self, x, y):
        return self

    def transform(self, x):
        return x.drop(self.cols, axis = 1)

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):

    def __init__(self, cols):
        self.cols = cols

    def fit(self, x, y):
        return self

    def transform(self, x):
        return x[self.cols]

In [ ]:
class RaceWrapper(BaseEstimator, ClassifierMixin):

    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, x, y):
        self.models = {}
        self.models['null'] = clone(self.estimator).fit(x[x.patient_race.isna()], y[x.patient_race.isna()])
        self.models['no_null'] = clone(self.estimator).fit(x[~x.patient_race.isna()], y[~x.patient_race.isna()])
        return self

    def predict_proba(self, x):
        prediction = np.zeros((len(x), 2))
        prediction[x.patient_race.isna()] = self.models['null'].predict_proba(x[x.patient_race.isna()])
        prediction[~x.patient_race.isna()] = self.models['no_null'].predict_proba(x[~x.patient_race.isna()])
        return prediction

In [ ]:
class BMIWrapper(BaseEstimator, ClassifierMixin):

    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, x, y):
        self.models = {}
        self.models['null'] = clone(self.estimator).fit(x[x.bmi.isna()], y[x.bmi.isna()])
        self.models['no_null'] = clone(self.estimator).fit(x[~x.bmi.isna()], y[~x.bmi.isna()])
        return self

    def predict_proba(self, x):
        prediction = np.zeros((len(x), 2))
        prediction[x.bmi.isna()] = self.models['null'].predict_proba(x[x.bmi.isna()])
        prediction[~x.bmi.isna()] = self.models['no_null'].predict_proba(x[~x.bmi.isna()])
        return prediction

In [ ]:
class ICDWrapper(BaseEstimator, ClassifierMixin):

    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, x, y):
        self.models = {}
        condition = x.breast_cancer_diagnosis_code.astype(str).str[0] == 'C'
        self.models['ICD10'] = clone(self.estimator).fit(x[condition], y[condition])
        self.models['ICD9'] = clone(self.estimator).fit(x[~condition], y[~condition])
        return self

    def predict_proba(self, x):
        prediction = np.zeros((len(x), 2))
        condition = x.breast_cancer_diagnosis_code.astype(str).str[0] == 'C'
        prediction[condition] = self.models['ICD10'].predict_proba(x[condition])
        prediction[~condition] = self.models['ICD9'].predict_proba(x[~condition])
        return prediction

In [ ]:
class RaceImputer(BaseEstimator, ClassifierMixin):

    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, x):

        train_data = x[~x.patient_race.isna()]
        target = x.pop('patient_race')

        self.le = LabelEncoder().fit(target)
        target = self.le.transform(target)

        self.estimator.fit(train_data, target)
        return self

    def transform(self, x):

        rain_data = x[x.patient_race.isna()]
        target = x.pop('patient_race')
        target = self.le.transform(target)

        prediction = self.le.transform(prediction)

In [ ]:
class Vectorizer(BaseEstimator, TransformerMixin):

    def __init__(self, max_features = 1000, cols = ['breast_cancer_diagnosis_desc'], n_components = 3):
        self.max_features = max_features
        self.cols = cols
        self.n_components = n_components

    def fit(self, x, y):
        self.vectorizer_dict = {}
        self.decomposer_dict = {}

        for col in self.cols:
            self.vectorizer_dict[col] = TfidfVectorizer(max_features = self.max_features).fit(x[col].astype(str), y)
            self.decomposer_dict[col] = TruncatedSVD(random_state = seed, n_components = self.n_components).fit(
                self.vectorizer_dict[col].transform(x[col].astype(str)), y
            )

        return self

    def transform(self, x):
        vectorized = {}

        for col in self.cols:
            vectorized[col] = self.vectorizer_dict[col].transform(x[col].astype(str))
            vectorized[col] = self.decomposer_dict[col].transform(vectorized[col])

        vectorized_df = pd.concat([pd.DataFrame(vectorized[col]).rename({
            f'truncatedsvd{i}' : f'{col}SVD{i}' for i in range(self.n_components)
        }, axis = 1) for col in self.cols], axis = 1)

        return pd.concat([x.reset_index(drop = True), vectorized_df], axis = 1)

In [ ]:
def cat_imputer(x):
    x_copy = x.copy()
    x_copy[list(x_copy.select_dtypes('object'))] = x_copy[list(x_copy.select_dtypes('object'))].fillna('Null').astype(str)
    return x_copy

CatImputer = FunctionTransformer(cat_imputer)

In [ ]:
def all_cat(x):
    x_copy = x.copy()
    x_copy['all_cat'] = ''
    for col in list(x.select_dtypes('object')) + ['patient_zip3'] if 'patient_zip3' in list(x) else []:
        x_copy['all_cat'] += x_copy[col].astype(str)
    return x_copy

AllCat = FunctionTransformer(all_cat)

In [ ]:
def null_indicator(x):
    x_copy = x.copy()
    x_copy['bmi_null'] = x_copy.bmi.isna().astype('object')
    x_copy['atmos_null'] = x_copy.Ozone.isna().astype('object')
    x_copy['size_null'] = x_copy.family_size.isna().astype('object')
    return x_copy

NullIndicator = FunctionTransformer(null_indicator)

In [ ]:
def float_rounder(x):
    x_copy = x.copy()
    x_copy.loc[:, x.select_dtypes('float').columns] = (x_copy.loc[:, x.select_dtypes('float').columns] * 1e6).fillna(0)
    x_copy = x_copy.astype({col : np.uint64 for col in x.select_dtypes('float').columns})
    return x_copy

FloatRounder = FunctionTransformer(float_rounder)

In [ ]:
def desc_cleaner(x):
    x_copy = x.copy()

    rep = {
        #'malig ' : 'malignant ',
        'malig ' : '',
        'malignant ' : '',
        #'neoplm' : 'neoplasm',
        'neoplm ' : '',
        'neoplasm ' : '',
        'unsp ' : 'unspecified ',
        'ovrlp' : 'overlapping',
        'sites' : 'site',
        'site,' : 'site',
        'breast,' : 'breast',
        'areola,' : 'areola',
        '(female),' : 'female',
        #' of ' : ' ',
        'of ' : '',
        ' and ' : ' ',
        '-' : ' '
    }
    rep = dict((re.escape(k), v) for k, v in rep.items())
    pattern = re.compile("|".join(rep.keys()))

    x_copy['breast_cancer_diagnosis_desc'] = x_copy.breast_cancer_diagnosis_desc.astype('str').apply(
        lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x.lower())
    )

    return x_copy

DescCleaner = FunctionTransformer(desc_cleaner)

In [ ]:
def desc_indicator(x):
    x_copy = x.copy()
    x_copy['breast_cancer_diagnosis_desc'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'upper' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'lower' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'inner' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'outer' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'axillary' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'breast' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'secondary' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'other' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'areola' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'overlapping' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'central' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'unspecified site' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'male' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'female' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'left' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'right' in x).astype(str) +\
                        x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'unspecified male' in x or 'unspecified female' in x).astype(str)
    return x_copy

DescIndicator = FunctionTransformer(desc_indicator)

In [ ]:
def desc_indicator_2(x):
    x_copy = x.copy()
    x_copy['upper'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'upper' in x)
    x_copy['lower'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'lower' in x)
    x_copy['inner'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'inner' in x)
    x_copy['outer'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'outer' in x)
    x_copy['axillary'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'axillary' in x)
    x_copy['breast'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'breast' in x)
    x_copy['secondary'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'secondary' in x)
    x_copy['other'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'other' in x)
    x_copy['areola'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'areola' in x)
    x_copy['overlapping'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'overlapping' in x)
    x_copy['central'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'central' in x)
    x_copy['unspecified'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'unspecified site' in x)
    x_copy['breast_male'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'male' in x)
    x_copy['breast_female'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'female' in x)
    x_copy['left'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'left' in x)
    x_copy['right'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'right' in x)
    x_copy['unspecified_lr'] = x_copy.breast_cancer_diagnosis_desc.astype(str).apply(lambda x: 'unspecified male' in x or 'unspecified female' in x)
    return x_copy.drop('breast_cancer_diagnosis_desc', axis = 1)

DescIndicator2 = FunctionTransformer(desc_indicator_2)

In [ ]:
def zip_splitter(x):
    x_copy = x.copy()
    x_copy['zip1'] = x_copy.patient_zip3.astype(str).str[0]
    x_copy['zip2'] = x_copy.patient_zip3.astype(str).str[1:]
    return x_copy

ZIPSplitter = FunctionTransformer(zip_splitter)

In [ ]:
def icd_switcher(x):
    x_copy = x.copy()
    x_copy['ICD10'] = (x_copy.breast_cancer_diagnosis_code.astype(str).str[0] == 'C').astype('object')
    return x_copy

ICDSwitcher = FunctionTransformer(icd_switcher)

In [ ]:
def metastatic_grouper(x):
    x_copy = x.copy()
    x_copy['metastatic_group'] = x.metastatic_cancer_diagnosis_code.astype(str).str[:3]
    return x_copy

MetastaticGrouper = FunctionTransformer(metastatic_grouper)

In [ ]:
def region_binner(x):
    x_copy = x.copy()
    x_copy['regional_age_bin'] = np.argmax(x[['age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80']], axis = 1)
    x_copy['regional_income_bin'] = np.argmax(x[['income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_50',
       'income_household_50_to_75', 'income_household_75_to_100',
       'income_household_100_to_150', 'income_household_150_over']], axis = 1)
    x_copy['regional_edu_bin'] = np.argmax(x[['education_less_highschool', 'education_highschool',
       'education_some_college', 'education_bachelors', 'education_graduate',
       'education_college_or_above', 'education_stem_degree']], axis = 1)
    return x_copy#.astype({'regional_age_bin' : 'object', 'regional_income_bin' : 'object', 'regional_edu_bin' : 'object'})

RegionBinner = FunctionTransformer(region_binner)

In [ ]:
def binner(x):
    x_copy = x.copy()
    x_copy['binned_age'] = (x_copy['patient_age'] / 2).astype(np.uint8)
    return x_copy#.astype({'binned_age' : 'object'})

Binner = FunctionTransformer(binner)

In [ ]:
Preprocessor = make_pipeline(DescCleaner, DescIndicator, ICDSwitcher, Binner, NullIndicator)

In [ ]:
def svd_rounder(x):

    x_copy = x.copy()
    for col in [column for column in list(x) if 'SVD' in column]:
        x_copy[col] = (x_copy[col] * 1e18).astype(np.int64)

    return x_copy

SVDRounder = FunctionTransformer(svd_rounder)

In [ ]:
ToNumpy = FunctionTransformer(lambda x: x.to_numpy())

In [ ]:
Sorter = FunctionTransformer(lambda x: x.sort_values(by = 'breast_cancer_diagnosis_code'))

# Model Cross Validation

In [ ]:
def cross_val_score(estimator, cv = skf, label = '', show_importance = False, pseudolabelling = False):

    X = train.copy()
    y = X.pop('DiagPeriodL90D')

    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(X)))
    train_scores, val_scores= [], []

    feature_importances_table = pd.DataFrame({'value' : 0}, index = list(X.columns))

    test_predictions = np.zeros((len(test)))

    #training model, predicting prognosis probability, and evaluating metrics
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):

        model = clone(estimator)

        #define train set
        X_train = X.iloc[train_idx].reset_index(drop = True)
        y_train = y.iloc[train_idx].reset_index(drop = True)

        #define validation set
        X_val = X.iloc[val_idx].reset_index(drop = True)
        y_val = y.iloc[val_idx].reset_index(drop = True)

        #train model
        model.fit(X_train, y_train)

        if pseudolabelling:
            pseudolabel = model.predict_proba(X_val).argmax(axis = 1)

            pseudo_X = pd.concat([X_train, X_val]).reset_index(drop = True)
            pseudo_y = pd.concat([y_train, pd.Series(pseudolabel)]).reset_index(drop = True)

            model.fit(pseudo_X, pseudo_y)

        #make predictions
        train_preds = model.predict_proba(X_train)[:, 1]
        val_preds = model.predict_proba(X_val)[:, 1]

        val_predictions[val_idx] += val_preds
        test_predictions += model.predict_proba(test)[:, 1] / cv.get_n_splits()
        if show_importance:
            feature_importances_table['value'] += permutation_importance(model, X_val, y_val, random_state = seed, scoring = make_scorer(roc_auc_score, needs_proba = True), n_repeats = 5).importances_mean / cv.get_n_splits()

        #evaluate model for a fold
        train_score = roc_auc_score(y_train, train_preds)
        val_score = roc_auc_score(y_val, val_preds)

        #append model score for a fold to list
        train_scores.append(train_score)
        val_scores.append(val_score)

    if show_importance:
        plt.figure(figsize = (20, 30))
        plt.title(f'Features with Biggest Importance of {np.mean(val_scores):.5f} ± {np.std(val_scores):.5f} Model', size = 25, weight = 'bold')
        sns.barplot(feature_importances_table.sort_values('value', ascending = False).T, orient = 'h', palette = 'viridis')
        plt.show()
    else:
        print(f'Val Score: {np.mean(val_scores):.5f} ± {np.std(val_scores):.5f} | Train Score: {np.mean(train_scores):.5f} ± {np.std(train_scores):.5f} | {label}')

    return val_scores, val_predictions, test_predictions

In [ ]:
score_list, oof_list, predict_list = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

# Feature Selector

In [ ]:
"""
Sequential feature selection
"""
from numbers import Integral, Real

import numpy as np

from sklearn.base import BaseEstimator, MetaEstimatorMixin, clone, is_classifier
from sklearn.metrics import get_scorer_names
from sklearn.model_selection import check_cv
from sklearn.utils._param_validation import HasMethods, Interval, StrOptions
from sklearn.utils._tags import _safe_tags
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_selection._base import SelectorMixin


class SequentialFeatureSelector(SelectorMixin, MetaEstimatorMixin, BaseEstimator):
    """Transformer that performs Sequential Feature Selection.

    This Sequential Feature Selector adds (forward selection) or
    removes (backward selection) features to form a feature subset in a
    greedy fashion. At each stage, this estimator chooses the best feature to
    add or remove based on the cross-validation score of an estimator. In
    the case of unsupervised learning, this Sequential Feature Selector
    looks only at the features (X), not the desired outputs (y).

    Read more in the :ref:`User Guide <sequential_feature_selection>`.

    .. versionadded:: 0.24

    Parameters
    ----------
    estimator : estimator instance
        An unfitted estimator.

    n_features_to_select : "auto", int or float, default="auto"
        If `"auto"`, the behaviour depends on the `tol` parameter:

        - if `tol` is not `None`, then features are selected while the score
          change does not exceed `tol`.
        - otherwise, half of the features are selected.

        If integer, the parameter is the absolute number of features to select.
        If float between 0 and 1, it is the fraction of features to select.

        .. versionadded:: 1.1
           The option `"auto"` was added in version 1.1.

        .. versionchanged:: 1.3
           The default changed from `"warn"` to `"auto"` in 1.3.

    tol : float, default=None
        If the score is not incremented by at least `tol` between two
        consecutive feature additions or removals, stop adding or removing.

        `tol` can be negative when removing features using `direction="backward"`.
        It can be useful to reduce the number of features at the cost of a small
        decrease in the score.

        `tol` is enabled only when `n_features_to_select` is `"auto"`.

        .. versionadded:: 1.1

    direction : {'forward', 'backward'}, default='forward'
        Whether to perform forward selection or backward selection.

    scoring : str or callable, default=None
        A single str (see :ref:`scoring_parameter`) or a callable
        (see :ref:`scoring`) to evaluate the predictions on the test set.

        NOTE that when using a custom scorer, it should return a single
        value.

        If None, the estimator's score method is used.

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

        - None, to use the default 5-fold cross validation,
        - integer, to specify the number of folds in a `(Stratified)KFold`,
        - :term:`CV splitter`,
        - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if the estimator is a classifier and ``y`` is
        either binary or multiclass,
        :class:`~sklearn.model_selection.StratifiedKFold` is used. In all other
        cases, :class:`~sklearn.model_selection.KFold` is used. These splitters
        are instantiated with `shuffle=False` so the splits will be the same
        across calls.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validation strategies that can be used here.

    n_jobs : int, default=None
        Number of jobs to run in parallel. When evaluating a new feature to
        add or remove, the cross-validation procedure is parallel over the
        folds.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    Attributes
    ----------
    n_features_in_ : int
        Number of features seen during :term:`fit`. Only defined if the
        underlying estimator exposes such an attribute when fit.

        .. versionadded:: 0.24

    feature_names_in_ : ndarray of shape (`n_features_in_`,)
        Names of features seen during :term:`fit`. Defined only when `X`
        has feature names that are all strings.

        .. versionadded:: 1.0

    n_features_to_select_ : int
        The number of features that were selected.

    support_ : ndarray of shape (n_features,), dtype=bool
        The mask of selected features.

    See Also
    --------
    GenericUnivariateSelect : Univariate feature selector with configurable
        strategy.
    RFE : Recursive feature elimination based on importance weights.
    RFECV : Recursive feature elimination based on importance weights, with
        automatic selection of the number of features.
    SelectFromModel : Feature selection based on thresholds of importance
        weights.

    Examples
    --------
    >>> from sklearn.feature_selection import SequentialFeatureSelector
    >>> from sklearn.neighbors import KNeighborsClassifier
    >>> from sklearn.datasets import load_iris
    >>> X, y = load_iris(return_X_y=True)
    >>> knn = KNeighborsClassifier(n_neighbors=3)
    >>> sfs = SequentialFeatureSelector(knn, n_features_to_select=3)
    >>> sfs.fit(X, y)
    SequentialFeatureSelector(estimator=KNeighborsClassifier(n_neighbors=3),
                              n_features_to_select=3)
    >>> sfs.get_support()
    array([ True, False,  True,  True])
    >>> sfs.transform(X).shape
    (150, 3)
    """

    _parameter_constraints: dict = {
        "estimator": [HasMethods(["fit"])],
        "n_features_to_select": [
            StrOptions({"auto"}),
            Interval(Real, 0, 1, closed="right"),
            Interval(Integral, 0, None, closed="neither"),
        ],
        "tol": [None, Interval(Real, None, None, closed="neither")],
        "direction": [StrOptions({"forward", "backward"})],
        "scoring": [None, StrOptions(set(get_scorer_names())), callable],
        "cv": ["cv_object"],
        "n_jobs": [None, Integral],
    }

    def __init__(
        self,
        estimator,
        *,
        n_features_to_select="auto",
        tol=None,
        direction="forward",
        scoring=None,
        cv=5,
        n_jobs=None,
    ):
        self.estimator = estimator
        self.n_features_to_select = n_features_to_select
        self.tol = tol
        self.direction = direction
        self.scoring = scoring
        self.cv = cv
        self.n_jobs = n_jobs

    #@_fit_context(
        # SequentialFeatureSelector.estimator is not validated yet
    #    prefer_skip_nested_validation=False
    #)
    def fit(self, X, y=None):
        """Learn the features to select from X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training vectors, where `n_samples` is the number of samples and
            `n_features` is the number of predictors.

        y : array-like of shape (n_samples,), default=None
            Target values. This parameter may be ignored for
            unsupervised learning.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        tags = self._get_tags()
        #X = self._validate_data(
        #    X,
        #    accept_sparse="csc",
        #    ensure_min_features=2,
        #    force_all_finite=not tags.get("allow_nan", True),
        #)
        n_features = X.shape[1]

        if self.n_features_to_select == "auto":
            if self.tol is not None:
                # With auto feature selection, `n_features_to_select_` will be updated
                # to `support_.sum()` after features are selected.
                self.n_features_to_select_ = n_features - 1
            else:
                self.n_features_to_select_ = n_features // 2
        elif isinstance(self.n_features_to_select, Integral):
            if self.n_features_to_select >= n_features:
                raise ValueError("n_features_to_select must be < n_features.")
            self.n_features_to_select_ = self.n_features_to_select
        elif isinstance(self.n_features_to_select, Real):
            self.n_features_to_select_ = int(n_features * self.n_features_to_select)

        if self.tol is not None and self.tol < 0 and self.direction == "forward":
            raise ValueError("tol must be positive when doing forward selection")

        cv = check_cv(self.cv, y, classifier=is_classifier(self.estimator))

        cloned_estimator = clone(self.estimator)

        # the current mask corresponds to the set of features:
        # - that we have already *selected* if we do forward selection
        # - that we have already *excluded* if we do backward selection
        current_mask = np.zeros(shape=n_features, dtype=bool)
        n_iterations = (
            self.n_features_to_select_
            if self.n_features_to_select == "auto" or self.direction == "forward"
            else n_features - self.n_features_to_select_
        )

        old_score = -np.inf
        is_auto_select = self.tol is not None and self.n_features_to_select == "auto"
        for _ in range(n_iterations):
            new_feature_idx, new_score = self._get_best_new_feature_score(
                cloned_estimator, X, y, cv, current_mask
            )
            if is_auto_select and ((new_score - old_score) < self.tol):
                break

            old_score = new_score
            current_mask[new_feature_idx] = True

        if self.direction == "backward":
            current_mask = ~current_mask

        self.support_ = current_mask
        self.n_features_to_select_ = self.support_.sum()

        return self

    def _get_best_new_feature_score(self, estimator, X, y, cv, current_mask):
        # Return the best new feature and its score to add to the current_mask,
        # i.e. return the best new feature and its score to add (resp. remove)
        # when doing forward selection (resp. backward selection).
        # Feature will be added if the current score and past score are greater
        # than tol when n_feature is auto,
        candidate_feature_indices = np.flatnonzero(~current_mask)
        scores = {}
        for feature_idx in candidate_feature_indices:
            candidate_mask = current_mask.copy()
            candidate_mask[feature_idx] = True
            if self.direction == "backward":
                candidate_mask = ~candidate_mask
            X_new = X.iloc[:, candidate_mask]
            scores[feature_idx] = cross_val_score_sfs(
                estimator,
                X_new,
                y,
                cv=cv,
                scoring=self.scoring,
                n_jobs=self.n_jobs,
            ).mean()
        new_feature_idx = max(scores, key=lambda feature_idx: scores[feature_idx])
        return new_feature_idx, scores[new_feature_idx]

    def _get_support_mask(self):
        check_is_fitted(self)
        return self.support_

    def _more_tags(self):
        return {
            "allow_nan": _safe_tags(self.estimator, key="allow_nan"),
        }

In [ ]:
def cross_val_score_sfs(estimator, X, y, cv, scoring, n_jobs):

    categorical_features = list(X.select_dtypes('object'))
    #if 'patient_zip3' in list(X):
    #    categorical_features += ['patient_zip3']

    scores = np.zeros((cv.get_n_splits()))

    if list(X) == ['patient_gender']:
        return np.array([.5])

    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):

        X_train = X.iloc[train_idx].reset_index(drop = True)
        y_train = y.iloc[train_idx].reset_index(drop = True)

        X_val = X.iloc[val_idx].reset_index(drop = True)
        y_val = y.iloc[val_idx].reset_index(drop = True)

        enc = MEstimateEncoder(cols = categorical_features).fit(X_train, y_train)
        X_train = enc.transform(X_train)
        X_val = enc.transform(X_val)

        if 'patient_zip3' in X.columns:
            cat_enc = CatBoostEncoder(cols = ['patient_zip3']).fit(X_train, y_train)
            X_train = cat_enc.transform(X_train)
            X_val = cat_enc.transform(X_val)

        model = clone(estimator).fit(X_train, y_train)

        #model = make_pipeline(
        #    CatImputer,
        #    CatBoostClassifier(random_state = seed, verbose = 0, cat_features = categorical_features)
        #).fit(X_train, y_train)

        scores[fold] = scoring(model, X_val, y_val)

    print(f'{scores.mean():.5f}: {list(X)}')

    return scores

In [ ]:
'''SequentialFeatureSelector(
    make_pipeline(SimpleImputer(), StandardScaler(), ScaledRidgeClassifier()),
    scoring = roc_scorer,
    tol = -1e-4,
    cv = skf,
    direction = 'backward'
).fit(
    (Preprocessor.transform(X)),
    y
).transform(
    (Preprocessor.transform(X)).fillna(0)
)'''

# Neural Network

In [ ]:
class TensorFlower(BaseEstimator, ClassifierMixin):

    def fit(self, x, y):

        tf.keras.utils.set_random_seed(seed)
        tf.config.experimental.enable_op_determinism()

        inputs = tf.keras.Input((x.shape[1],))
        x0 = tf.keras.layers.BatchNormalization()(inputs)

        x1 = tf.keras.layers.Dense(64)(x0)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        x1 = tf.keras.layers.LeakyReLU()(x1)

        x2 = tf.keras.layers.Dense(16)(x1)
        x2 = tf.keras.layers.BatchNormalization()(x2)
        x2 = tf.keras.layers.LeakyReLU()(x2)

        x3 = tf.keras.layers.Dense(4)(x2)
        x3 = tf.keras.layers.BatchNormalization()(x3)
        x3 = tf.keras.layers.LeakyReLU()(x3)

        x4 = tf.keras.layers.Dense(1)(x3)
        x4 = tf.keras.layers.BatchNormalization()(x4)
        outputs = tf.keras.activations.sigmoid(x4)

        self.model = tf.keras.Model(inputs, outputs)
        self.model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.AdamW(1e-4))

        self.model.fit(x, y, epochs = 50, verbose = 0)
        self.classes_ = np.unique(y)

        return self

    def predict_proba(self, x):

        probabilities = np.zeros((len(x), 2))
        probabilities[:, 1] = self.model.predict(x, verbose = 0)[:, 0]
        probabilities[:, 0] = 1 - probabilities[:, 1]
        return probabilities

    def predict(self, x):

        return np.argmax(self.predict_proba(x), axis = 1)


In [ ]:
TensorFlowey = make_pipeline(
    Preprocessor,
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'bmi_null', 'atmos_null', 'size_null']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    FeatureSelector(['payer_type', 'patient_age', 'metastatic_cancer_diagnosis_code', 'age_40s', 'age_50s', 'age_over_80', 'family_size', 'income_household_75_to_100', 'home_value', 'education_less_highschool', 'education_some_college', 'self_employed', 'limited_english', 'commute_time', 'health_uninsured', 'ICD10', 'binned_age', 'bmi_null', 'size_null']),
    SimpleImputer(),
    TensorFlower()
)

#_, oof_list['TF'], predict_list['TF'] = cross_val_score(TensorFlowey, show_importance = False)

# SVC

In [ ]:
SVM = make_pipeline(
    Preprocessor,
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'bmi_null', 'atmos_null', 'size_null']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    CatBoostEncoder(cols = ['patient_zip3']),
    FeatureSelector(['payer_type', 'patient_zip3', 'patient_age', 'bmi', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'Region', 'Division', 'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married', 'widowed', 'family_size', 'family_dual_income', 'income_household_median', 'income_household_under_5', 'income_household_5_to_10', 'income_household_10_to_15', 'income_household_15_to_20', 'income_household_20_to_25', 'income_household_25_to_35', 'income_household_35_to_50', 'income_household_50_to_75', 'income_household_75_to_100', 'income_household_100_to_150', 'income_household_150_over', 'income_household_six_figure', 'income_individual_median', 'home_ownership', 'housing_units', 'home_value', 'rent_median', 'rent_burden', 'education_less_highschool', 'education_highschool', 'education_some_college', 'education_bachelors', 'education_graduate', 'education_college_or_above', 'education_stem_degree', 'labor_force_participation', 'unemployment_rate', 'self_employed', 'farmer', 'race_white', 'race_black', 'race_asian', 'race_native', 'race_pacific', 'race_other', 'race_multiple', 'hispanic', 'disabled', 'poverty', 'limited_english', 'commute_time', 'health_uninsured', 'veteran', 'Ozone', 'PM25', 'N02', 'bmi_null', 'atmos_null', 'size_null']),
    SimpleImputer(),
    StandardScaler(),
    SVC(random_state = seed, probability = True, max_iter = 10000000)
)

_, oof_list['SVM'], predict_list['SVM'] = cross_val_score(SVM)

# GaussianNB

In [ ]:
GNB = make_pipeline(
    Preprocessor,
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'bmi_null', 'atmos_null', 'size_null']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    FeatureSelector(['payer_type', 'patient_age', 'metastatic_cancer_diagnosis_code', 'age_40s', 'age_50s', 'age_over_80', 'family_size', 'income_household_75_to_100', 'home_value', 'education_less_highschool', 'education_some_college', 'self_employed', 'limited_english', 'commute_time', 'health_uninsured', 'ICD10', 'binned_age', 'bmi_null', 'size_null']),
    SimpleImputer(),
    StandardScaler(),
    GaussianNB()
)

#_, oof_list['GNB'], predict_list['GNB'] = cross_val_score(GNB)

# BernoulliNB

In [ ]:
BNB = make_pipeline(
    Preprocessor,
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'bmi_null', 'atmos_null', 'size_null']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division']),
    FeatureSelector(['payer_type', 'patient_age', 'bmi', 'metastatic_cancer_diagnosis_code', 'Region', 'age_30s', 'age_50s', 'age_60s', 'family_size', 'income_household_under_5', 'income_household_35_to_50', 'home_ownership', 'education_less_highschool', 'self_employed', 'race_white', 'race_pacific', 'race_other', 'ICD10', 'binned_age', 'bmi_null', 'size_null']),
    SimpleImputer(),
    StandardScaler(),
    BernoulliNB()
)

#_, oof_list['BNB'], predict_list['BNB'] = cross_val_score(BNB)

# Log

In [ ]:
Log = make_pipeline(
    Preprocessor,
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'bmi_null', 'atmos_null', 'size_null']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    FeatureSelector(['payer_type', 'patient_age', 'metastatic_cancer_diagnosis_code', 'age_median', 'age_40s', 'age_60s', 'never_married', 'family_size', 'family_dual_income', 'income_household_under_5', 'income_household_10_to_15', 'income_household_25_to_35', 'income_household_35_to_50', 'income_household_50_to_75', 'income_household_six_figure', 'housing_units', 'home_value', 'rent_burden', 'education_less_highschool', 'education_stem_degree', 'self_employed', 'race_white', 'race_black', 'race_asian', 'race_multiple', 'commute_time', 'health_uninsured', 'Ozone', 'ICD10', 'bmi_null']),
    SimpleImputer(),
    StandardScaler(),
    LogisticRegression(random_state = seed, max_iter = 1000000000)
)

_, oof_list['Log'], predict_list['Log'] = cross_val_score(Log, show_importance = False)

# RidgeClassifier

In [ ]:
class ScaledRidgeClassifier(BaseEstimator, ClassifierMixin):

    def fit(self, x, y):
        model = RidgeClassifier(random_state = seed).fit(x, y)
        self.weight = model.coef_[0]
        self.bias = model.intercept_[0]
        self.classes_ = np.unique(y)
        return self

    def predict_proba(self, x):
        predictions = np.zeros((len(x), 2))
        predictions[:, 1] = MinMaxScaler().fit_transform(
            (x.to_numpy() @ self.weight + self.bias).reshape((-1, 1))
        ).x0
        predictions[:, 0] = 1 - predictions[:, 1]
        return predictions

    def predict(self, x):
        return np.argmax(self.predict_proba(x), axis = 1)

In [ ]:
Ridger = make_pipeline(
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division']),
    CatBoostEncoder(cols = ['patient_zip3']),
    SimpleImputer(),
    ScaledRidgeClassifier()
)

_, oof_list['Ridge'], predict_list['Ridge'] = cross_val_score(Ridger)

# LDA

In [ ]:
LDA = ICDWrapper(make_pipeline(
    Preprocessor,
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'bmi_null', 'atmos_null', 'size_null']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    FeatureSelector(['payer_type', 'patient_age', 'metastatic_cancer_diagnosis_code', 'age_median', 'age_40s', 'age_60s', 'never_married', 'family_size', 'family_dual_income', 'income_household_under_5', 'income_household_10_to_15', 'income_household_25_to_35', 'income_household_35_to_50', 'income_household_50_to_75', 'income_household_six_figure', 'housing_units', 'home_value', 'rent_burden', 'education_less_highschool', 'education_highschool', 'education_some_college', 'education_college_or_above', 'education_stem_degree', 'self_employed', 'race_white', 'race_black', 'race_asian', 'race_multiple', 'commute_time', 'health_uninsured', 'Ozone', 'ICD10', 'binned_age', 'bmi_null']),
    SimpleImputer(),
    #StandardScaler(),
    LinearDiscriminantAnalysis()
))

_, oof_list['LDA'], predict_list['LDA'] = cross_val_score(LDA, show_importance = False)

# K-Neighbors

In [ ]:
KNN = make_pipeline(
    Preprocessor,
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'bmi_null']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division']),
    FeatureSelector(['patient_race', 'payer_type', 'patient_age', 'bmi', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'Region', 'Division', 'density', 'age_median', 'age_under_10', 'age_10_to_19', 'age_40s', 'age_60s', 'age_70s', 'age_over_80', 'male', 'female', 'married', 'divorced', 'widowed', 'income_household_5_to_10', 'income_household_15_to_20', 'income_household_20_to_25', 'income_household_50_to_75', 'income_household_75_to_100', 'housing_units', 'education_less_highschool', 'education_some_college', 'education_graduate', 'education_stem_degree', 'unemployment_rate', 'farmer', 'race_white', 'race_asian', 'race_native', 'race_multiple', 'poverty', 'limited_english', 'commute_time', 'health_uninsured', 'veteran', 'ICD10', 'binned_age', 'bmi_null']),
    SimpleImputer(),
    StandardScaler(),
    KNeighborsClassifier(100)
)

_, oof_list['KNN'], predict_list['KNN'] = cross_val_score(KNN)

# Random Forest

In [ ]:
RF = make_pipeline(
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    FeatureSelector(['payer_type', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code']),
    SimpleImputer(),
    RandomForestClassifier(random_state = seed, criterion = 'log_loss', n_estimators = 100)
)

_, oof_list['RF'], predict_list['RF'] = cross_val_score(RF)

# Extra Trees

In [ ]:
ET = make_pipeline(
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    FeatureSelector(['breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment_type']),
    SimpleImputer(),
    ExtraTreesClassifier(random_state = seed, criterion = 'log_loss')
)

_, oof_list['ET'], predict_list['ET'] = cross_val_score(ET)

# AdaBoost

In [ ]:
Ada = make_pipeline(
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    FeatureSelector(['payer_type', 'patient_age', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'Region', 'income_household_under_5', 'income_household_15_to_20', 'home_value', 'unemployment_rate', 'race_pacific']),
    SimpleImputer(),
    AdaBoostClassifier(random_state = seed)
)

#_, oof_list['Ada'], predict_list['Ada'] = cross_val_score(Ada)

# Gradient Boosting Classifier

In [ ]:
def gb_objective(trial):
    params = {
        'learning_rate' : trial.suggest_float('learning_rate', .001, .1, log = True),
        'max_depth' : trial.suggest_int('max_depth', 2, 20),
        'subsample' : trial.suggest_float('subsample', .1, 1),
        'min_samples_split' : trial.suggest_int('min_samples_split', 2, 50),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 2, 50),
        'n_estimators' : 500,
        'random_state' : seed,
    }

    optuna_model = make_pipeline(
        MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']),
        MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
        FeatureSelector(['payer_type', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'Region', 'Division', 'rent_median', 'race_other']),
        SimpleImputer(),
        GradientBoostingClassifier(**params)
    )

    optuna_score, _, _ = cross_val_score(optuna_model)

    return np.mean(optuna_score)

gb_study = optuna.create_study(direction = 'maximize')

In [ ]:
#gb_study.optimize(gb_objective, 200)

In [ ]:
gb_params = {'learning_rate': 0.01113789617580569, 'max_depth': 5, 'subsample': 0.7141679039385381, 'min_samples_split': 30, 'min_samples_leaf': 13}

GB = make_pipeline(
    MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']),
    MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    FeatureSelector(['payer_type', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'Region', 'Division', 'rent_median', 'race_other']),
    SimpleImputer(),
    GradientBoostingClassifier(random_state = seed)
)

_, oof_list['GB'], predict_list['GB'] = cross_val_score(GB)

# Hist Gradient Boosting Classifier

In [ ]:
def hgb_objective(trial):
    params = {
        'learning_rate' : trial.suggest_float('learning_rate', .001, .1, log = True),
        'max_depth' : trial.suggest_int('max_depth', 2, 20),
        'max_leaf_nodes' : trial.suggest_int('max_leaf_nodes', 2, 50),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 5, 50),
        'l2_regularization' : trial.suggest_float('l2_regularization', .1, 20, log = True),
        'max_iter' : 750,
        'random_state' : seed,
    }

    optuna_model = make_pipeline(
        MEstimateEncoder(categorical_features),
        FeatureSelector(['payer_type', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'Region', 'age_under_10', 'rent_median']),
        SimpleImputer(),
        HistGradientBoostingClassifier(**params)
    )

    optuna_score, _, _ = cross_val_score(optuna_model)

    return np.mean(optuna_score)

hgb_study = optuna.create_study(direction = 'maximize')

In [ ]:
#hgb_study.optimize(hgb_objective, 200)

In [ ]:
hgb_params = {'learning_rate': 0.004309668685467947, 'max_depth': 11, 'max_leaf_nodes': 35, 'min_samples_leaf': 6, 'l2_regularization': 3.335157086503191}

HGB = make_pipeline(
    #MEstimateEncoder(cols = ['patient_gender', 'payer_type', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']),
    #MEstimateEncoder(cols = ['patient_race', 'patient_state', 'Region', 'Division'], handle_missing = 'return_nan'),
    MEstimateEncoder(categorical_features),
    FeatureSelector(['payer_type', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'Region', 'age_under_10', 'rent_median']),
    SimpleImputer(),
    HistGradientBoostingClassifier(**hgb_params, max_iter = 750, random_state = seed)
)

_, oof_list['HGB'], predict_list['HGB'] = cross_val_score(HGB)

# XGBoost

In [ ]:
def xgb_objective(trial):
    params = {
        'eta' : trial.suggest_float('eta', .001, .3, log = True),
        'max_depth' : trial.suggest_int('max_depth', 2, 30),
        'subsample' : trial.suggest_float('subsample', .5, 1),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', .1, 1),
        'min_child_weight' : trial.suggest_float('min_child_weight', .1, 20, log = True),
        'reg_lambda' : trial.suggest_float('reg_lambda', .01, 20, log = True),
        'reg_alpha' : trial.suggest_float('reg_alpha', .01, 10, log = True),
        'n_estimators' : 750,
        'random_state' : seed,
        'tree_method' : 'hist',
    }

    optuna_model = make_pipeline(
        MEstimateEncoder(cols = categorical_features),
        FeatureSelector(cols = ['payer_type', 'patient_state', 'patient_age', 'patient_gender', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code']),
        XGBClassifier(**params)
    )

    optuna_score, _, _ = cross_val_score(optuna_model)

    return np.mean(optuna_score)

xgb_study = optuna.create_study(direction = 'maximize')

In [ ]:
#xgb_study.optimize(xgb_objective, 200)

In [ ]:
xgb_params = {'eta': 0.014290460861843577, 'max_depth': 10, 'subsample': 0.891507291340268, 'colsample_bytree': 0.3779121278340872, 'min_child_weight': 0.15457089929898238, 'reg_lambda': 15.22147437968379, 'reg_alpha': 0.9406055353007685}

XGB = make_pipeline(
    MEstimateEncoder(cols = categorical_features),
    FeatureSelector(cols = ['payer_type', 'patient_state', 'patient_age', 'patient_gender', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code']),
    XGBClassifier(**xgb_params, random_state = seed, n_estimators = 750, tree_method = 'hist')
)

_, oof_list['XGB'], predict_list['XGB'] = cross_val_score(XGB, show_importance = False)

# XGBoost Random Forest

In [ ]:
XGBRF = make_pipeline(
    MEstimateEncoder(cols = categorical_features),
    FeatureSelector(cols = ['patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'Region', 'education_highschool', 'race_other']),
    XGBRFClassifier(random_state = seed, n_estimators = 100)
)

#_, oof_list['XGBRF'], predict_list['XGBRF'] = cross_val_score(XGBRF, show_importance = False)

# LightGBM

In [ ]:
def lgb_objective(trial):
    params = {
        'learning_rate' : trial.suggest_float('learning_rate', .001, .1, log = True),
        'max_depth' : trial.suggest_int('max_depth', 2, 20),
        'subsample' : trial.suggest_float('subsample', .5, 1),
        'min_child_weight' : trial.suggest_float('min_child_weight', .1, 15, log = True),
        'reg_lambda' : trial.suggest_float('reg_lambda', .1, 20, log = True),
        'reg_alpha' : trial.suggest_float('reg_alpha', .1, 10, log = True),
        'n_estimators' : 600,
        'random_state' : seed,
        'boosting_type' : 'dart'
    }

    optuna_model = make_pipeline(
        MEstimateEncoder(cols = categorical_features),
        FeatureSelector(cols = ['patient_age', 'patient_gender', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'Region', 'age_20s', 'disabled']),
        LGBMClassifier(**params)
    )

    optuna_score, _, _ = cross_val_score(optuna_model)

    return np.mean(optuna_score)

lgb_study = optuna.create_study(direction = 'maximize')

In [ ]:
#lgb_study.optimize(lgb_objective, 200)

In [ ]:
lgb_params = {'learning_rate': 0.005807018851897142, 'max_depth': 18, 'subsample': 0.7421423838804505, 'min_child_weight': 0.6370978748964402, 'reg_lambda': 2.554455122943933, 'reg_alpha': 0.37469472538903054}

LGB = make_pipeline(
    MEstimateEncoder(cols = categorical_features),
    FeatureSelector(cols = ['patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment_type', 'Region', 'unemployment_rate', 'disabled']),
    LGBMClassifier(**lgb_params, random_state = seed, n_estimators = 600)
)

_, oof_list['LGB'], predict_list['LGB'] = cross_val_score(LGB, show_importance = False)

# LightGBM DART

In [ ]:
dart_params = {'learning_rate': 0.03689647027034999, 'max_depth': 8, 'subsample': 0.7558778300751996, 'min_child_weight': 0.930494385854166, 'reg_lambda': 1.7436925731064161, 'reg_alpha': 0.30133492167125675}

DART = make_pipeline(
    MEstimateEncoder(cols = categorical_features),
    FeatureSelector(cols = ['patient_age', 'patient_gender', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'Region', 'age_20s', 'disabled']),
    LGBMClassifier(**dart_params, random_state = seed, n_estimators = 600, boosting_type = 'dart')
)

#_, oof_list['DART'], predict_list['DART'] = cross_val_score(DART, show_importance = False)

# CatBoost

In [ ]:
def cb_objective(trial):
    params = {
        'learning_rate' : trial.suggest_float('learning_rate', .001, .1, log = True),
        'max_depth' : trial.suggest_int('max_depth', 2, 12),
        'subsample' : trial.suggest_float('subsample', 0.01, 1, log = True),
        'colsample_bylevel' : trial.suggest_float('colsample_bylevel', .1, 1),
        'min_child_samples' : trial.suggest_int('min_child_samples', 1, 15),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.01, 1, log = True),
        'n_estimators' : 1000,
        'bootstrap_type' : 'Bernoulli',
        'random_state' : seed,
        'verbose' : 0,
        'has_time' : True
    }

    cb_features = ['patient_race', 'payer_type', 'patient_state', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'age_under_10', 'unemployment_rate', 'self_employed']
    cb_cat = ['patient_race', 'payer_type', 'patient_state', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code']

    optuna_model = make_pipeline(
        CatImputer,
        FeatureSelector(cb_features),
        CatBoostClassifier(**params, cat_features = cb_cat)
    )

    optuna_score, _, _ = cross_val_score(optuna_model)

    return np.mean(optuna_score)

cb_study = optuna.create_study(direction = 'maximize')

In [ ]:
#cb_study.optimize(cb_objective, 200)

In [ ]:
cb_features = ['payer_type', 'patient_zip3', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'family_size', 'self_employed']
cb_cat = ['payer_type', 'patient_zip3', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code']

cb_params = {'learning_rate': 0.015116075947127675, 'max_depth': 5, 'colsample_bylevel': 0.5236776088105062, 'min_child_samples': 15, 'reg_lambda': 0.8666134907089579}

CB = make_pipeline(
    CatImputer,
    FeatureSelector(cols = cb_features),
    CatBoostClassifier(random_state = seed, verbose = 0, cat_features = cb_cat)
)

_, oof_list['CB'], predict_list['CB'] = cross_val_score(CB)

In [ ]:
cb_time_features = ['payer_type', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'income_household_35_to_50']
cb_time_cat = ['payer_type', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code']

cb_time_params = {'learning_rate': 0.02460510521563778, 'max_depth': 4, 'colsample_bylevel': 0.5614033622786926, 'min_child_samples': 9, 'reg_lambda': 0.04325296956640457}

CB_Time = make_pipeline(
    CatImputer,
    FeatureSelector(cols = cb_time_features),
    CatBoostClassifier(random_state = seed, verbose = 0, cat_features = cb_time_cat, has_time = True)
)

_, oof_list['CB_Time'], predict_list['CB_Time'] = cross_val_score(CB_Time)

# CatBoost (Bayesian)

In [ ]:
cb_bayes_features = ['payer_type', 'patient_state', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'self_employed', 'health_uninsured']
cb_bayes_cat = ['payer_type', 'patient_state', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code']

cb_bayes_params = {'learning_rate': 0.02023191659311952, 'max_depth': 5, 'bagging_temperature': 0.014373761190553567, 'colsample_bylevel': 0.5372235775413032, 'min_child_samples': 11, 'reg_lambda': 0.7584977523122014}

CB_Bayes = make_pipeline(
    CatImputer,
    FeatureSelector(cols = cb_bayes_features),
    CatBoostClassifier(random_state = seed, verbose = 0, bootstrap_type = 'Bayesian', cat_features = cb_bayes_cat)
)

_, oof_list['CB_Bayes'], predict_list['CB_Bayes'] = cross_val_score(CB_Bayes)

In [ ]:
cb_bayes_time_features = ['payer_type', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'income_household_50_to_75', 'health_uninsured']
cb_bayes_time_cat = ['payer_type', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code']

CB_Bayes_Time = make_pipeline(
    CatImputer,
    FeatureSelector(cols = cb_bayes_time_features),
    CatBoostClassifier(random_state = seed, verbose = 0, bootstrap_type = 'Bayesian', cat_features = cb_bayes_time_cat, has_time = True)
)

_, oof_list['CB_Bayes_Time'], predict_list['CB_Bayes_Time'] = cross_val_score(CB_Bayes_Time)

# CatBoost (Bernoulli)

In [ ]:
cb_bernoulli_features = ['payer_type', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'health_uninsured']
cb_bernoulli_cat = ['payer_type', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code']

cb_bernoulli_params = {'learning_rate': 0.017997837369237192, 'max_depth': 5, 'subsample': 0.7652481186657251, 'colsample_bylevel': 0.5001460413426887, 'min_child_samples': 8, 'reg_lambda': 0.8862964454272922}

CB_Bernoulli = make_pipeline(
    CatImputer,
    FeatureSelector(cols = cb_bernoulli_features),
    CatBoostClassifier(random_state = seed, verbose = 0, bootstrap_type = 'Bernoulli', cat_features = cb_bernoulli_cat)
)

_, oof_list['CB_Bernoulli'], predict_list['CB_Bernoulli'] = cross_val_score(CB_Bernoulli)

In [ ]:
cb_bernoulli_time_features = ['patient_race', 'payer_type', 'patient_state', 'patient_age', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code', 'age_under_10', 'unemployment_rate', 'self_employed']
cb_bernoulli_time_cat = ['patient_race', 'payer_type', 'patient_state', 'breast_cancer_diagnosis_code', 'metastatic_cancer_diagnosis_code']

cb_bernoulli_time_params = {'learning_rate': 0.01799872581282759, 'max_depth': 5, 'subsample': 0.8869608354825106, 'colsample_bylevel': 0.8751857544649823, 'min_child_samples': 1, 'reg_lambda': 0.4461074278850297}

CB_Bernoulli_Time = make_pipeline(
    CatImputer,
    FeatureSelector(cols = cb_bernoulli_time_features),
    CatBoostClassifier(**cb_bernoulli_time_params, random_state = seed, verbose = 0, bootstrap_type = 'Bernoulli', cat_features = cb_bernoulli_time_cat, has_time = True)
)

_, oof_list['CB_Bernoulli_Time'], predict_list['CB_Bernoulli_Time'] = cross_val_score(CB_Bernoulli_Time)

# Ensemble

In [ ]:
def multiplier_objective(trial):

    multipliers = np.array([trial.suggest_float(f'{label}', -1, 1) for label in list(oof_list)])

    return roc_auc_score(train.DiagPeriodL90D, oof_list.to_numpy() @ (multipliers / multipliers.sum()))

multiplier_study = optuna.create_study(direction = 'maximize')

In [ ]:
#multiplier_study.optimize(multiplier_objective, 25000)

In [ ]:
#selected_model = [col] + ['CB_Bernoulli_Time', 'CB_Bernoulli', 'Log', 'TF', 'ET', 'SVM', 'CB', 'RF', 'LGB', 'KNN', 'CB_Time', 'CB_Bayes_Time', 'CB_Bayes', 'GNB', 'BNB', 'LDA', 'XGB', 'GB']
#weights = RidgeClassifier(random_state = seed).fit(oof_list, train.DiagPeriodL90D).coef_[0]
#weights = np.array(list(multiplier_study.best_params.values()))
weights = np.array(list({'SVM': 0.5371288937104951, 'Log': -0.9998838102461505, 'Ridge': 0.9033487720861355, 'LDA': -0.5526242685240101, 'KNN': -0.30108472639200856, 'RF': 0.13224519493772863, 'ET': -0.46569815350021637, 'GB': 0.3996602359300103, 'HGB': -0.3247882797025673, 'XGB': 0.3561482377729332, 'LGB': -0.41235352612324705, 'CB': -0.4094488354882769, 'CB_Time': -0.3300533530466142, 'CB_Bayes': 0.1974948592239418, 'CB_Bayes_Time': 0.3424885322288681, 'CB_Bernoulli': -0.8637868599495759, 'CB_Bernoulli_Time': -0.8766136385461583}.values()))
weights /= weights.sum()
pd.DataFrame(weights, index = list(oof_list), columns = ['weight per model'])

In [ ]:
print(f'Score: {roc_auc_score(train.DiagPeriodL90D, oof_list.to_numpy() @ weights)}')
predictions = predict_list.to_numpy() @ weights

# Submission

In [ ]:
submission = test.copy()
submission['DiagPeriodL90D'] = predictions

submission.DiagPeriodL90D.to_csv('submission.csv')

In [ ]:
plt.figure(figsize = (15, 10), dpi = 300)
sns.kdeplot(predictions, fill = True)
plt.title("Distribution of DiagPeriodL90D Probability", weight = 'bold', size = 25)
plt.show()

Thanks for reading!